In [155]:
import json

import pandas as pd
from pandas import json_normalize
import datetime

with open("./data2.json") as f:
    json_data = json.load(f)["classes"]

df = pd.DataFrame(json_data)
df = df[
    ["crse_id", "class_section", "enrollment_total", "instruction_mode", "strm", "subject", "subject_descr", "meetings"]
]

df = df.explode("meetings")
meetings = json_normalize(df["meetings"])
meetings = meetings[["days", "start_time", "end_time"]]
df = df.drop(columns=["meetings"]).reset_index(drop=True)
df = pd.concat([df, meetings], axis=1)

df = df[df.days != "TBA"]

df.start_time = pd.to_datetime(df.start_time, format="%H.%M.%S.000000").dt.time
df.end_time = pd.to_datetime(df.end_time, format="%H.%M.%S.000000").dt.time

df["mo"] = df.days.str.contains("Mo")
df["tu"] = df.days.str.contains("Tu")
df["we"] = df.days.str.contains("We")
df["th"] = df.days.str.contains("Th")
df["fr"] = df.days.str.contains("Fr")

df.drop(columns=["days"], inplace=True)

min_time = pd.Timestamp(df.start_time.min().strftime("%H:%M:%S"))
max_time = pd.Timestamp(df.end_time.max().strftime("%H:%M:%S"))
time_range = pd.date_range(min_time, max_time, freq="1min").time
timeSlotTable = pd.DataFrame(time_range).rename(columns={0: "time"})

termTable = pd.DataFrame(df.strm.unique())
weekdayTable = pd.DataFrame(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]).rename(columns={0: "weekday"})

courseTable = df[["crse_id", "subject"]].drop_duplicates().rename(columns={"crse_id": "courseId", "subject": "major"})
sessionTable = df[["crse_id", "strm", "class_section", "instruction_mode"]].reset_index(drop=True).rename(columns= {"crse_id": "courseId", "strm": "termId", "class_section": "section", "instruction_mode": "instructionMode"})
occupancy = df[["strm", ]]

display(sessionTable)

df.to_csv("data.csv", index=False)

,courseId,termId,section,instructionMode
0,000014,2252,01,P
1,000014,2252,02,P
2,000025,2252,01,P
3,000025,2252,02,P
4,000025,2252,03,P
...,...,...,...,...
64,000206,2252,03,P
65,022587,2252,01,P
66,000229,2252,03,P
67,000229,2252,04,P
